## Building a Chatbot

In this, we'll go over an example of how to design and implent an LLM -  powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that I build will only use the language model to have a converstion. There are several other related concepts that we may be looking for.

- Conversational RAG: Enable a chatbit exprience over an external source of data.
- Agents: Build a chatbot that can take actions.

This will helps adavanced topics.

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()  # loading all the env variables

groq_api_key = os.getenv("GROQ_API_KEY")


In [7]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key = groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001E80B83BE80>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001E80B83BD30>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Aman and I am a AI Engineer.")])

AIMessage(content="Nice to meet you, Aman. It's great to hear that you're an AI Engineer. AI is a fascinating field with endless possibilities. What kind of projects or areas of AI are you currently working on or interested in? Are you into machine learning, natural language processing, computer vision, or something else? I'm here to chat and help with any questions or topics you'd like to discuss.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 49, 'total_tokens': 131, 'completion_time': 0.136380368, 'completion_tokens_details': None, 'prompt_time': 0.002493192, 'prompt_tokens_details': None, 'queue_time': 0.050264972, 'total_time': 0.13887356}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbb03-dd58-7941-934f-428a8154f0d1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_to

In [9]:
## Checking - Remembring the previous context..
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Aman and I am a AI Engineer."),
        AIMessage(content='Nice to meet you, Aman. As an AI Engineer, you must be working on developing and implementing artificial intelligence models, integrating them with other systems, and ensuring their accuracy and efficiency.'),
        HumanMessage(content="Hey, what is my name and What do I do?")
    ]
)

AIMessage(content='Your name is Aman, and you are an AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 109, 'total_tokens': 123, 'completion_time': 0.019749247, 'completion_tokens_details': None, 'prompt_time': 0.006555566, 'prompt_tokens_details': None, 'queue_time': 0.049569424, 'total_time': 0.026304813}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbb03-debb-7061-a53a-c44ae6be9f39-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 109, 'output_tokens': 14, 'total_tokens': 123})

## Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future intracttions will then load those messages and pass them into the chain as part of the input.

In [10]:
!pip install langchain_community

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

## store the chat-session
store = {}

## Separate the different sessions - This will distinguish one chat session with other
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id] 


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [12]:
## test with dummy config
config = {'configurable': {"session_id": "chat1"}}

In [13]:
## COmbining this chat session with this session Id -  Chat1
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Aman and I am  a AI Engineer.")],
    config=config
)

In [14]:
response.content

'Nice to meet you, Aman. As an AI Engineer, you must be working on exciting projects involving the development and implementation of artificial intelligence and machine learning models. What specific areas of AI are you interested in or working on? Are you into natural language processing, computer vision, or perhaps reinforcement learning?'

In [15]:
## session remembering the data with session id
with_message_history.invoke(
    [HumanMessage(content="What is my name ?")],
    config=config
)


AIMessage(content='Your name is Aman.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 126, 'total_tokens': 133, 'completion_time': 0.004924655, 'completion_tokens_details': None, 'prompt_time': 0.008295168, 'prompt_tokens_details': None, 'queue_time': 0.049779951, 'total_time': 0.013219823}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbb03-f0e7-76c1-b5a9-76271dee5744-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 126, 'output_tokens': 7, 'total_tokens': 133})

In [16]:
## Change the config --- changing the sessionId
config1 = {'configurable': {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name ?")],
    config=config1
)

response.content

"I don't have any information about your name. I'm a large language model, I don't have the ability to know your personal details unless you choose to share them with me. If you'd like to tell me your name, I'd be happy to chat with you and learn more about you."

In [17]:
## Now adding info with sessionId -  chat2
response = with_message_history.invoke(
    [HumanMessage(content = "My name is Steve.")],
    config=config1
)

In [18]:
config1 = {'configurable': {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content = "What is my name ?")],
    config=config1
)

response.content

'Your name is Steve. You mentioned it earlier.'

## Prompt Templates

Prompt Templates help to trun raw user information into a information into a format that the LLM can work with.
In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking message as input). Next, add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpfu assistant. Answer all the question to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [20]:
chain.invoke({"messages": [HumanMessage(content="Hi, My name is Aman.")]})

AIMessage(content="Nice to meet you, Aman. I'm happy to assist you with any questions or topics you'd like to discuss. How's your day going so far?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 61, 'total_tokens': 95, 'completion_time': 0.050615272, 'completion_tokens_details': None, 'prompt_time': 0.003406641, 'prompt_tokens_details': None, 'queue_time': 0.051965799, 'total_time': 0.054021913}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bbb03-f449-7ab1-b23b-b7aede513446-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 61, 'output_tokens': 34, 'total_tokens': 95})

In [21]:
## Run with session_id
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [22]:
config3 = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Aman.")],
    config = config3
)

response.content

'Nice to meet you, Aman. How can I assist you today?'

In [23]:
## Adding more complexity

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpfu assistant. Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [24]:
response = chain.invoke(
    {
    "messages": [HumanMessage(content="Hi My name is Aman.")],
    "language": "Hindi"
    }
)

response.content

'नमस्ते Aman जी, मैं आपकी सहायता करने के लिए यहां हूँ। क्या आपके पास कोई प्रश्न है, जिस पर मैं आपकी सहायता कर सकता हूँ?'

Now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [25]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [26]:
config4 = {"configurable": {"session_id": "chat4"}}

In [27]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content = "Hi, I am Aman.")],
     "language" : "Hindi"},
    config = config4
)

response.content

'नमस्ते Aman, मैं आपकी मदद करने के लिए यहाँ हूँ! क्या आप मुझसे कुछ पूछना चाहते हैं या किसी विशिष्ट विषय पर चर्चा करना चाहते हैं?'

## Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmannaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM, Therefore, it is important to add a step that limits the size of the messages that we are passing in.

Trim_Message: It is helper to reduce how many messages we are sending to the model. the trimmer allows us to specify how many tokens we want to keeo, along with other parameters like if we want to always keep the system messages and whether to allow partail messages.

In [36]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [ ]:
## Set of messages
messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi, I'm Bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="What 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]

In [45]:
## trimmed the conversation
trimmer.invoke(messages)

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='What 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]

In [46]:
## In chain, how to pass trimmer function
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

In [47]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do I like?")],
        "language" : "English"
    }
)

In [ ]:
## Trimmed the message conversation
response.content

"I'm a text-based AI assistant, I don't have any information about your personal preferences or tastes. We just started our conversation, and I don't have any prior knowledge about you. Would you like to share what your favorite ice cream flavor is?"

In [49]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what maths problem I asked for?")],
        "language" : "English"
    }
)

response.content

'You asked me to solve the math problem 2 + 2.'

In [56]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config5 = {"congigurable": {"session_id": "chat5"}}

In [57]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language" : "English"
    },
    config=config5,
)

response.content

"I don't know your name, you didn't tell me."

In [58]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what maths problem I asked for?")],
        "language" : "English"
    },
    config=config5,
)

response.content

'You asked me a simple math problem: 2 + 2.'